In [1]:
import pandas as pd
import os

env = os.environ.get('Environment')
print(f"Environment: {env!r}")

Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

In [3]:
%%athena_to_df --out distinct_last_tag_id
SELECT 
    tag_id,
    MAX(ts) AS last_ts
FROM "prod_plant_connectivity_prepared_data"."tag_timeserie"
GROUP BY 1

In [13]:
%%athena_to_df --out distinct_metadata
SELECT DISTINCT 
    SUBSTRING(tag_id,1,3) AS site_id,
    building_id,
    source_name,
    tag_id
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"

In [31]:
df_merged = distinct_metadata.merge(distinct_last_tag_id, how='left', on='tag_id')

In [33]:
df_ordered = df_merged.sort_values('last_ts', ascending=False)
tag_id_with_ts = df_ordered.drop_duplicates(subset=['site_id','building_id','source_name'], keep='first')
tag_id_without_ts = df_ordered[df_ordered['last_ts'].isnull()]
unique_tag_id = pd.concat([tag_id_with_ts, tag_id_without_ts])
unique_tag_id

,site_id,building_id,source_name,tag_id,last_ts
8521,SIP,SCPP,ns=2;s=SCP.PLC 2.pcnr_TT1062,SIPSCPP-EST002-PEC0144-TRT1062.PEC-TRM-TEM-CTR,2023-05-24 12:59:59
18071,MLE,00HG,ns=2;s=Modbus.BARexpert.Grd.133,MLE00HG-WAT001-PDL0001-FQRC001.WAT-DIS-FLO-QUA,2023-05-24 12:59:59
20446,SIP,ENOX,ns=2;s=Enox.PLC 16.pcnr_PDIA280941,SIPENOX-UTI001-DYI0001-PRC0941.SPD-PRD-PRE-CTR,2023-05-24 12:59:59
27286,SCO,CHIH,ns=2;s=Allen-Bradley_ControlLogix_Energy_Monit...,SCOCHIH-ELS001-CHI0003-JIT0001.ELE-LVS-ENE-CTR,2023-05-24 12:59:59
20774,SIP,SCPP,ns=2;s=SCP.PLC 3.pcnr_PT1014,SIPSCPP-PUR003-CEN0101-PT01014.CEN-TRM-PRE-CTR,2023-05-24 12:59:59
...,...,...,...,...,...
38608,SIP,ENOX,ns=2;s=Enox.PLC 11.pcnr_PIX21103,SIPENOX-TNF001-DSS1200-PC21103.PEC-PRD-PRE-CTR,NaT
38611,MLE,00T5,ns=2;s=PlantCo T5 Ambr250 Modular 137117.Ambr2...,MLE00T5-CUC002-BIO0003-FEE0007.CUC-BIO-STA-SET,NaT
38614,MLE,00T5,ns=2;s=PlantCo T5 Ambr250 Modular 137117.Ambr2...,MLE00T5-CUC002-BIO0001-OXP0002.CUC-BIO-RAT-FCT,NaT
38618,MLE,00T5,ns=2;s=PlantCo T5 OPC UA.Ambr250.ambr250HT.Bio...,MLE00T5-CUC001-BIO0005-OXP0001.CUC-BIO-FLO-MOD,NaT


In [34]:
non_compliant_tags_due2format = unique_tag_id.loc[unique_tag_id['tag_id'].str.count('-') > 6, ['site_id', 'tag_id']]
non_compliant_tags_due2format = non_compliant_tags_due2format.drop_duplicates(subset='tag_id')
non_compliant_tags_due2format.to_csv('data/non_compliant_tags_due2format.csv', index=False)